<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-08-11 22:50:11


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*10)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100[:]
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)
my_prospects = list(df_mypf[(df_mypf['LatestQtr'] == 1) & df_mypf['StarStock'] == 1]['Symbol'].values)

In [6]:
def print_distribution():

  # star stocks
  print('Overall - Star Stocks:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('Overall - V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Overall - Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Overall - Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

  # stocks in portfolio
  print('Portfolio - Stocks:')
  print(df_mypf['InPortfolio'].value_counts())
  print('-------')

  # star stocks in portfolio
  print('Portfolio - Star Stocks:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())
  print('-------')

  # latest quarter in portfolio
  print('Portfolio - Latest Quarter:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['LatestQtr'].value_counts())
  print('-------')

  # criteria of portfolio stocks
  print('Portfolio - Criteria:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())
  print('-------')

  # strategies in portfolio stocks
  print('Portfolio - Strategy:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())
  print('-------')

  # conviction distribution
  print('Portfolio - Conviction:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Conviction'].value_counts())

print_distribution()

Overall - Star Stocks:
1.0    96
0.0    91
Name: StarStock, dtype: int64
-------
Overall - V40/V40N Stocks:
V40N    39
V40     38
VR      26
V200     6
AR       4
SS       1
Name: Criteria, dtype: int64
-------
Overall - Latest Qtr:
1    154
0     34
Name: LatestQtr, dtype: int64
-------
Overall - Star + V40/V40N Stocks:
1.0    43
0.0    34
Name: StarStock, dtype: int64
-------
Portfolio - Stocks:
0    132
1     56
Name: InPortfolio, dtype: int64
-------
Portfolio - Star Stocks:
1.0    32
0.0    24
Name: StarStock, dtype: int64
-------
Portfolio - Latest Quarter:
1    47
0     9
Name: LatestQtr, dtype: int64
-------
Portfolio - Criteria:
VR      26
V40     12
V40N    11
AR       4
V200     3
Name: Criteria, dtype: int64
-------
Portfolio - Strategy:
ATH     39
2T3Y    14
BTT      3
Name: Strategy, dtype: int64
-------
Portfolio - Conviction:
M    30
L    15
H    11
Name: Conviction, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='10y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      tmp['Investment'] = tmp['Avg_Cost'] * tmp['Shares']
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-14 00:00:00+05:30,19564.0,18035.0,8.48,^NSEI,19979.0,73.0
2023-07-21 00:00:00+05:30,19745.0,18105.0,9.06,^NSEI,19979.0,67.0
2023-07-28 00:00:00+05:30,19646.0,18167.0,8.14,^NSEI,19979.0,60.0
2023-08-04 00:00:00+05:30,19517.0,18229.0,7.07,^NSEI,19979.0,52.0
2023-08-11 00:00:00+05:30,19428.0,18281.0,6.28,^NSEI,19979.0,48.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

#df_common_features['Stock'] = df_common_features['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_common_features['Discount%PE'] = round((df_common_features['Med_PE']-df_common_features['Curr_PE'])*100/(df_common_features['Med_PE']),0)
df_common_features['Target'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
df_common_features['Upside%Target'] = round((df_common_features['Target']-df_common_features['Close'])*100/(df_common_features['Close']),0)
df_common_features = df_common_features.set_index('Stock')

In [13]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Upside%Target',	'RSI_14', 'Discount%PE', 'Curr_ROCE%', 'Curr_ROE%', 'Conviction',  'LatestQtr', 'StarStock', 'Criteria', 'Strategy', 'Investment']
tmp_df = df_common_features[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))

investment = round(sum(tmp_df['Investment']),0)
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 56


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,,
VALIANTORG.NS,471.0,-11.95,293.0,32.0,32.0,14.80,13.00,L,1,0.0,VR,2T3Y,98532.85,2.70
JPPOWER.NS,7.0,6.43,271.0,70.0,-2284.0,5.16,0.55,L,1,0.0,VR,ATH,30960.00,0.85
PNB.NS,63.0,20.29,238.0,60.0,6.0,4.11,3.34,M,1,1.0,VR,ATH,76958.00,2.11
BANDHANBNK.NS,227.0,-2.22,216.0,51.0,20.0,6.67,11.90,M,1,0.0,VR,2T3Y,97808.00,2.68
NIACL.NS,126.0,12.23,188.0,59.0,15.0,4.86,4.06,L,0,0.0,VR,ATH,45657.30,1.25
LUXIND.NS,1583.0,5.69,187.0,59.0,-62.0,12.70,9.95,M,0,0.0,V40N,2T3Y,76179.72,2.09
GLAND.NS,1648.0,20.47,162.0,85.0,23.0,14.80,10.90,L,1,0.0,VR,2T3Y,67422.85,1.85
METROPOLIS.NS,1379.0,1.42,149.0,46.0,1.0,17.40,15.20,L,1,0.0,VR,ATH,30051.56,0.82
VAIBHAVGBL.NS,388.0,22.27,149.0,75.0,-58.0,11.10,9.02,L,1,0.0,VR,2T3Y,45717.12,1.25


In [14]:
# top 5 from portfolio to book profit based on Upside%ATH
tmp_df.sort_values(by = 'Upside%Target', ascending=True).head(7)

,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,,
TCS.NS,3449.0,5.76,12.0,57.0,-2.0,59.1,46.9,H,1,1.0,V40,ATH,51696.00,1.42
UJJIVANSFB.NS,50.0,60.65,14.0,70.0,-5852.0,1000.0,1000.0,M,1,1.0,V40N,ATH,68265.00,1.87
PIDILITIND.NS,2545.0,1.34,14.0,38.0,1.0,24.6,18.7,H,1,1.0,V40,ATH,75121.80,2.06
EICHERMOT.NS,3389.0,1.50,14.0,51.0,36.0,27.4,21.1,H,1,1.0,V40N,ATH,48903.00,1.34
EMAMILTD.NS,520.0,26.12,15.0,82.0,33.0,28.1,28.3,M,1,1.0,V200,ATH,30426.48,0.83
ICICIGI.NS,1371.0,15.76,18.0,53.0,18.0,21.2,17.6,H,1,1.0,V40,ATH,98916.72,2.71
5PAISA.NS,456.0,38.24,21.0,64.0,58.0,12.4,10.6,M,1,1.0,V40N,ATH,79129.68,2.17


In [15]:
# top 10 stocks from portfolio for SIP based on 200 DMA and RSI
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'RSI_14', ascending=True).head(10)

,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy,Investment,Weightage%
Stock,,,,,,,,,,,,,,
DMART.NS,3550.0,-4.41,50.0,32.0,40.0,20.10,16.0,M,1,1.0,AR,ATH,51232.44,1.40
VALIANTORG.NS,471.0,-11.95,293.0,32.0,32.0,14.80,13.0,L,1,0.0,VR,2T3Y,98532.85,2.70
KOTAKBANK.NS,1791.0,-2.74,27.0,35.0,37.0,6.86,14.2,L,1,1.0,V40,BTT,50854.70,1.39
TEAMLEASE.NS,2298.0,-4.70,130.0,41.0,38.0,14.80,15.1,M,1,1.0,V40N,2T3Y,47878.53,1.31
FINEORG.NS,4604.0,-8.65,56.0,46.0,39.0,65.30,49.4,M,1,0.0,V40N,ATH,51671.73,1.42
BANDHANBNK.NS,227.0,-2.22,216.0,51.0,20.0,6.67,11.9,M,1,0.0,VR,2T3Y,97808.00,2.68
INFY.NS,1372.0,-2.36,35.0,52.0,20.0,40.70,31.8,H,1,1.0,V40,ATH,137703.28,3.78
RAJESHEXPO.NS,527.0,-19.56,80.0,52.0,33.0,10.30,10.5,M,0,1.0,V40N,ATH,72182.50,1.98
IEX.NS,128.0,-7.55,124.0,56.0,23.0,53.00,40.7,M,1,1.0,VR,ATH,50156.20,1.38


In [16]:
#df_common_features.drop([ '200_SMA', 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_common_features[~df_common_features.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)][cols]
tmp_base_df.drop(['Investment'], axis=1, inplace=True)

In [17]:
# latest quarter prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(my_prospects)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,


In [18]:
# star + v40/v40n prospects (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
MOTILALOFS.NS,915.0,36.18,61.0,75.0,52.0,13.0,15.6,L,1,1.0,V40N,NaN
ASIANPAINT.NS,3185.0,4.71,33.0,29.0,25.0,35.2,27.7,H,1,1.0,V40,BTT
KANSAINER.NS,329.0,15.44,33.0,58.0,NaN,14.5,10.7,M,1,1.0,V40N,NaN
BAJFINANCE.NS,7061.0,7.83,30.0,38.0,34.0,11.8,23.5,H,1,1.0,V40,BTT
DIXON.NS,4781.0,30.37,20.0,69.0,NaN,25.8,22.4,M,1,1.0,V40N,NaN


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
TITAN.NS,3017.0,13.86,4.0,57.0,-7.0,25.10,30.8,H,1,1.0,V40,NaN
CAPLIPOINT.NS,1032.0,41.05,3.0,70.0,NaN,26.80,22.4,M,1,1.0,V40N,NaN
ICICIBANK.NS,953.0,6.46,3.0,47.0,26.0,6.35,17.2,H,1,1.0,V40,NaN
POLYCAB.NS,4659.0,47.68,3.0,68.0,-34.0,27.10,20.0,M,1,1.0,V40N,NaN
MARICO.NS,577.0,12.69,1.0,63.0,-4.0,43.00,35.8,M,1,1.0,V40,NaN
3MINDIA.NS,29469.0,23.35,0.0,62.0,NaN,31.40,23.4,M,1,1.0,V40N,NaN
CERA.NS,8424.0,32.61,0.0,71.0,-19.0,25.80,19.4,H,1,1.0,V40N,NaN


In [20]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] >= 25)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
ADANITRANS.NS,832.0,-46.41,393.0,59.0,NaN,10.50,11.6,L,1,1.0,NaN,NaN
ADANIGREEN.NS,971.0,-23.52,206.0,43.0,NaN,7.81,22.8,L,1,1.0,NaN,NaN
GODREJIND.NS,500.0,11.36,36.0,62.0,55.0,9.75,11.2,L,0,1.0,NaN,NaN
LICI.NS,660.0,7.47,31.0,66.0,NaN,144.00,125.0,M,1,1.0,V200,NaN
SBICARD.NS,867.0,7.41,31.0,51.0,NaN,13.00,25.7,M,1,1.0,NaN,NaN
CANBK.NS,332.0,11.07,28.0,52.0,47.0,5.33,14.8,H,1,1.0,NaN,NaN
IGL.NS,444.0,0.17,27.0,32.0,NaN,27.20,21.1,L,1,1.0,NaN,NaN


In [21]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
#print(tmp_df.index)
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
TITAN.NS,3017.0,13.86,4.0,57.0,-7.0,25.10,30.8,H,1,1.0,V40,NaN
CAPLIPOINT.NS,1032.0,41.05,3.0,70.0,NaN,26.80,22.4,M,1,1.0,V40N,NaN
ICICIBANK.NS,953.0,6.46,3.0,47.0,26.0,6.35,17.2,H,1,1.0,V40,NaN
POLYCAB.NS,4659.0,47.68,3.0,68.0,-34.0,27.10,20.0,M,1,1.0,V40N,NaN
MARICO.NS,577.0,12.69,1.0,63.0,-4.0,43.00,35.8,M,1,1.0,V40,NaN
3MINDIA.NS,29469.0,23.35,0.0,62.0,NaN,31.40,23.4,M,1,1.0,V40N,NaN
CERA.NS,8424.0,32.61,0.0,71.0,-19.0,25.80,19.4,H,1,1.0,V40N,NaN


In [22]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
NESTLEIND.NS,21842.0,6.33,6.0,34.0,NaN,138.0,108.0,H,0,0.0,V40,NaN
PGHH.NS,15284.0,8.52,5.0,49.0,NaN,110.0,79.5,L,0,0.0,V40,NaN


In [23]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,


In [24]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
JCHAC.NS,1038.0,-6.16,210.0,59.0,NaN,1000.00,1000.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,889.0,-5.48,131.0,56.0,NaN,14.80,13.50,M,1,0.0,V40N,NaN
LALPATHLAB.NS,2298.0,6.64,82.0,47.0,NaN,18.50,15.00,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1542.0,8.61,73.0,74.0,-29.0,8.78,6.37,M,1,0.0,V40,NaN
SFL.NS,1144.0,-3.42,72.0,44.0,NaN,13.60,12.50,M,1,0.0,V40N,NaN
MOTILALOFS.NS,915.0,36.18,61.0,75.0,52.0,13.00,15.60,L,1,1.0,V40N,NaN
SIS.NS,445.0,14.18,52.0,58.0,NaN,10.60,15.60,M,1,0.0,V40N,NaN
PFIZER.NS,3965.0,-0.52,51.0,56.0,-223.0,27.20,19.80,L,0,0.0,V40,NaN
TATAELXSI.NS,7114.0,5.14,49.0,34.0,NaN,60.00,39.30,M,0,0.0,V40N,NaN


In [25]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%Target'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Upside%Target,RSI_14,Discount%PE,Curr_ROCE%,Curr_ROE%,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,
JCHAC.NS,1038.0,-6.16,210.0,59.0,NaN,1000.00,1000.00,M,1,0.0,V40N,NaN
SYMPHONY.NS,889.0,-5.48,131.0,56.0,NaN,14.80,13.50,M,1,0.0,V40N,NaN
LALPATHLAB.NS,2298.0,6.64,82.0,47.0,NaN,18.50,15.00,M,1,0.0,V40N,NaN
WHIRLPOOL.NS,1542.0,8.61,73.0,74.0,-29.0,8.78,6.37,M,1,0.0,V40,NaN
SFL.NS,1144.0,-3.42,72.0,44.0,NaN,13.60,12.50,M,1,0.0,V40N,NaN
MOTILALOFS.NS,915.0,36.18,61.0,75.0,52.0,13.00,15.60,L,1,1.0,V40N,NaN
SIS.NS,445.0,14.18,52.0,58.0,NaN,10.60,15.60,M,1,0.0,V40N,NaN
PFIZER.NS,3965.0,-0.52,51.0,56.0,-223.0,27.20,19.80,L,0,0.0,V40,NaN
TATAELXSI.NS,7114.0,5.14,49.0,34.0,NaN,60.00,39.30,M,0,0.0,V40N,NaN


In [26]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_common_features[df_common_features.index == stock_id]

,Close,200_SMA,Dev%_200,Max,RSI_14,Prev_Close,Avg_Cost,Shares,Med_PE,Curr_PE,...,Curr_ROE%,Criteria,Strategy,Target,LatestQtr,StarStock,Conviction,Investment,Discount%PE,Upside%Target
Stock,,,,,,,,,,,,,,,,,,,,,
PVRINOX.NS,1639.0,1596.0,2.68,1917.0,75.0,1631.0,NaN,NaN,51.2,1000.0,...,1000.0,NaN,NaN,1917.0,0,0.0,L,NaN,-1853.0,17.0


In [27]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [28]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-07-13 00:00:00+05:30,658.0,603.0,9.20,BERGEPAINT.NS,854.0,46.0
2023-07-20 00:00:00+05:30,681.0,604.0,12.81,BERGEPAINT.NS,854.0,63.0
2023-07-27 00:00:00+05:30,671.0,606.0,10.72,BERGEPAINT.NS,854.0,51.0
2023-08-03 00:00:00+05:30,695.0,608.0,14.36,BERGEPAINT.NS,854.0,61.0
2023-08-11 00:00:00+05:30,701.0,610.0,14.87,BERGEPAINT.NS,854.0,60.0
